# **Injertos Tipográficos 🈸🌾**

Este es el cuaderno base que se utilizó durante el Laboratorio *Injertos Tipográficos*, proyecto ganador​ ​de​ ​la​ ​Beca​ Plataforma Bogotá en Arte, Ciencia y Tecnología ​2021.

Por favor, antes de empezar, **asegurarse de hacer una copia de los archivos de [esta carpeta](https://drive.google.com/drive/folders/1dU99WNiC8ytsxTVL_bN_8kKZ5SIf8lO_?usp=sharing) o hacer una copia de este cuaderno en su cuenta de Google Drive.**

Si hace una copia de la carpeta, este cuaderno se encontrará dentro de /ShapeMatchingGAN/src.


---
## Montar Drive
Todos los archivos que se utilizan para crear los injertos (imágenes, modelos, scripts) estarán alojados en el drive de cada usuario. Para vincular Colab con Drive, es necesario ejecutar la siguiente celda:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importar carpeta **src**

Más adelante se van a utilizar archivos que requieren importar la carpeta **src**, donde están alojados. Está incluida en la carpeta que se pidió copiar al principio del cuaderno.

**Copiar la ruta de la carpeta usando el explorador de archivos de la izquierda y pegarlo en la siguiente celda, en la línea que empieza con `sys.path.append`.**

La ruta a la carpeta debe terminar en '/ShapeMatchingGAN/src'

Por ejemplo: '/content/drive/MyDrive/S6_220521/ShapeMatchingGAN/src'

In [ ]:
import sys
# Copiar ruta a la carpeta en la siguiente linea
sys.path.append('/content/drive/MyDrive/Maleza/S6_220521/ShapeMatchingGAN/src')

---
# **Creación de datasets**

Un Injerto Tipográfico se crea trasladando la forma y la textura de una imagen estilo a una letra usando la herramienta [Shape-MatchingGAN](https://williamyang1991.github.io/projects/ICCV2019/SMGAN.html). Para que SMGAN aprenda a trasladar estas características de una imágen a otra, hay que entrenarla con unos datasets específicos.

![injerto](https://i.imgur.com/SqZWpba.jpg)

## ImageMagick

Para crear los datasets necesarios se puede utilizar [ImageMagick](https://imagemagick.org/index.php), un software gratuito que permite crear, editar, componer y convertir imágenes digitales. Se utiliza mediante comandos (lineas de texto), por ejemplo: `magick image.jpg image.png`

El software tiene varios componentes entre ellos `magick`, `convert`, `montage`y `mirage`. En este cuaderno se utilizan algunos de ellos.

Lo primero que se debe hacer es instalar IM ejecutando la siguiente celda:

In [ ]:
!sudo apt-get update && sudo apt-get upgrade && apt install imagemagick

Luego se puede confirmar la versión de ImageMagick que está instalada:

In [ ]:
!convert -version

## Base tipográfica
El primer dataset que se crea es el de texto. SMGAN requiere aprender a reconocer la forma de letras u otros símbolos para poder trasladar las características de la imagen de estilo. El proceso detallado se encuentra en la sección *4.1 Bidirectional Structure Transfer (GS)* del [paper de SMGAN](https://arxiv.org/abs/1905.01354).

  **Importante: Si ya se han creado las imagenes de texto anteriormente se puede pasar al paso** ***Aumento de dataset***, **asegurándose que:**

- **Las imágenes NO tengan canal de transparencia. Más Información [aquí](https://sking7.github.io/articles/66662475.html).**
- **Las imágenes estén preferiblemente en formato .jpg.**
- **Las imaágenes tengan el mismo tamaño que la imágen de estilo.**

El comando que nos permite crear una imagen con texto en blanco y negro es el siguiente:

```
convert -gravity center -background "rgb(0,0,0)" -fill "rgb(255,255,255)" -define colorspace:auto-grayscale=off -type TrueColor -alpha off -size 500x500 -font "ruta/a/la/fuente.ttf" -pointsize 280 caption:Texto ruta/a/la_imagen.jpg
```

- `convert` llama a la librería ImageMagick.

- `-gravity center` obliga a centrar el texto dentro de la imagen.

- `-background "rgb(0,0,0)"` establece un fondo de color negro.

- `-define colorspace:auto-grayscale=off` y `-type TrueColor` obligan a que las imágenes no estén en escala de grises. Esto tiene invonvenientes con los scripts de SMGAN.

- `-alpha off` establece que las imágenes no tengan transparencia.

- `-size 500x500` establece el tamaño de las imágenes en 500 pixeles de ancho y de alto. **Importante: el tamaño de estas imágenes debe ser igual al tamaño de la imágen de estilo.**

- `-font "ruta/a/la/fuente.ttf"` establece que fuente usar.

- `-pointsize 280` establece el tamaño de la fuente.

- `caption:Texto` establece el texto que se dibujará.

- `ruta/a/la_imagen.jpg` establece a donde se guarda la imagen y el nombre del archivo.

Luego de ejecutarlo, el resultado es el siguiente:

![resultado](https://i.imgur.com/iC9F5k8.jpg)

Como con otras GANs, es mejor utilizar una gran cantidad de imágenes para su entrenamiento. En la carpeta /Glifos/base/original se encuentra el archivo `base_original.sh` que tiene un comando que repite el procedimiento anterior varias veces con las diferentes letras del abecedario. **Usando el explorador de la izquierda puede buscar el archivo y hacer doble click para editarlo y cambiar los ajustes.**

```
array=(A B C)
for i in "${array[@]}"
do
	convert -gravity center -background "rgb(0,0,0)" -fill "rgb(255,255,255)" -define colorspace:auto-grayscale=off -type TrueColor -alpha off -size 500x500 -font "ruta/a/la/fuente.ttf" -pointsize 280 caption:$i ruta/a/las_imagenes/$i'.jpg'
done
echo "${#array[@]} imágenes creadas."

```

Con las siguientes celdas

1. Se le indica a Colab que vaya a la carpeta de Drive que contiene el script **(cambiar la ruta a la carpeta de acuerdo a donde la haya guardado):**

In [ ]:
import os
%cd '/content/drive/MyDrive/S6_220521/Glifos/base/original'

2. Se ejecuta el script

In [ ]:
%%shell
bash '/content/drive/MyDrive/S6_220521/Glifos/base_original.sh'

Este es un ejemplo de los resultados que se obtienen al ejecutar la celdas anteriores:

![ejemplo_a](https://i.imgur.com/v6Uhy7y.jpg)

## Aumento de dataset
Para que el entrenamiento sea más efectivo, se debe aumentar el dataset con imágenes con diferentes transformaciones. Esto se hace con la libreria [Augmentor](https://github.com/mdbloice/Augmentor) de python. Las documentación de las principales transformaciones se puede encontrar [aquí](https://augmentor.readthedocs.io/en/master/userguide/mainfeatures.html).

Lo primero que se debe hacer es instalar la librería:

In [ ]:
# Instalar la librería Augmentor
!pip install Augmentor

Luego se aplican las transformaciones. Tener en cuenta los ajustes:

- `carpeta_original` es donde se encuentran las imágenes en blanco y negro creadas anteriormente.
- `carpeta_aumentada` es donde se guardarán las nuevas imágenes transformadas.
- `cant_imagenes` es la cantidad de imágenes que se va a crear.
- `rot_probability`, `rot_left` y `rot_right` definen que tan probable es que las imágenes se roten y cuántos grados en cada dirección.
- `zoom_probability`, `zoom_min` y `zoom_max` definen qué tan probable es que se cambie el tamaño de las imágenes y en qué cantidad.
- `flip_horizontal_probability` y `flip_vertical_probability` define qué tan probable es que las imágenes sean volteadas horizontal y verticalmente (espejo).

In [ ]:
# Importa la librería
import Augmentor

# Rutas de las carpetas con las imágenes originales y donde se guardan las aumentadas.
carpeta_original = "/content/drive/MyDrive/S6_220521/Glifos/base/original" #@param {type:"string"}
carpeta_aumentada = "/content/drive/MyDrive/S6_220521/Glifos/base/original/augmentor" #@param {type:"string"}

# Cantidad de imágenes a crear:
cant_imagenes =  520#@param {type : "integer"}

# Variables para controlar las transformaciones
# Rotación. Los valores de rot_left y rot_right se da en grados.
rot_probability = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
rot_left = 12 #@param {type:"slider", min:0, max:25, step:1}
rot_right = 12 #@param {type:"slider", min:0, max:25, step:1}

# Zoom
zoom_probability = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
zoom_min = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
zoom_max = 1.8 #@param {type:"slider", min:0, max:2, step:0.1}

# Reflejo:
flip_horizontal_probability = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}
flip_vertical_probability = 0.5 #@param {type:"slider", min:0.1, max:1, step:0.1}

# Iniciar la librería
p = Augmentor.Pipeline(carpeta_original, output_directory=carpeta_aumentada)

# Opciones de las ransformaciones
p.rotate(probability=rot_probability, max_left_rotation=rot_left, max_right_rotation=rot_right)
p.zoom(probability=zoom_probability, min_factor=zoom_min, max_factor=zoom_max)
p.flip_left_right(probability=flip_horizontal_probability)
p.flip_top_bottom(probability=flip_vertical_probability)

# Cantidad de imágenes
p.sample(cant_imagenes)

Al ejecutar el script quedarán guardadas imágenes como la siguiente:

![letra_aumentada](https://i.imgur.com/6llo3FO.png)

## Imágenes de distancia

Para entrenar la red es necesario convertir las imágenes blanco y negro a imágenes basadas en distancia. Estas se crean al calcular la distancia entre los diferentes bordes que existen en la imagen. Según [el repositorio](https://github.com/VITA-Group/ShapeMatchingGAN#testing-example) de SMGAN, las imágenes basadas en distancia hacen que la red se ocupe mejor de las regiones saturadas.

La siguiente celda genera estas versiones usando las imágenes creadas en el paso anterior. 

- `carpeta_augmentor` es donde se guardaron las imágenes transformadas con Augmentor.
- `carpeta_train` es donde se guardarán las nuevas imágenes basadas en distancia.


In [ ]:
# Importa las librerias necesarias
import os
import glob
import utils

# Variable para numerar las imágenes
image_num = 0

# Rutas de las carpetas de augmentor y train
carpeta_augmentor = "/content/drive/MyDrive/S6_220521/Glifos/base/original/augmentor" #@param {type:"string"}
carpeta_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train" #@param {type:"string"}

# Indicar que se usen los archivos .jpg en la carpeta augmentor
carpeta_augmentor_files = carpeta_augmentor + "/*.jpg"

# Crea las versiones basadas en distancia
for image in glob.glob(carpeta_augmentor_files):
    utils.text_image_preprocessing(image, carpeta_train + "/" + str(image_num) + ".png")
    image_num += 1

Finalmente la siguiente celda re-nombra las imágenes para que sigan el patrón requerido por SMGAN: `0000.png, 0001.png, 0002.png, ...` **Importante: cambiar la ruta a la `carpeta_train` usada anteriormente en la línea que comienza con `os.chdir('')`**

In [ ]:
# Importa las librerías necesarias
import os
import glob

# Ruta a la carpeta con las imágenes creadas anteriormente. No olvidar cambiar.
os.chdir(r"/content/drive/MyDrive/S6_220521/Glifos/base/train")
for index, oldfile in enumerate(glob.glob("*.png"), start=0):
    newfile = '{:04}.png'.format(index)
    os.rename (oldfile,newfile)

Este es el tipo de imágenes finales que se usarán para entrenar la SMGAN:

![distance](https://i.imgur.com/M2lstx4.png)

## **Imágen de estilo**
El segundo componente de los injertos es la imágen de estilo. De esta imagen se obtendrá la forma y la textura de los Injertos. **Es necesario que previamente se haya escogido una imagen y escalado al mismo tamaño de las imágenes de estilo:**

![original](https://i.imgur.com/Jxycvxa.png)

y que se haya hecho una máscara en blanco y negro separando la planta y el fondo. Lo que se encuentre de blanco será lo que se utilizará para deformar y texturizar las letras.

![mask](https://i.imgur.com/2Oufmgg.png)

Ambas imágenes se deben guardar en la carpeta **Estilo**. En este caso solo se necesita una imagen por lo que el proceso se puede hacer usando Photoshop, GIMP o cualquier otro editor.

Con las siguientes celdas
1. Se crea la versión basada en distancia de la imagen blanco y negro.

  - `carpeta_estilo` es la ruta de la carpeta donde se guardaron las dos imágenes.
  - `imagen_BN` es el nombre del archivo de las máscara en blanco y negro.
  - `imagen_DISTANCIA` es el nombre que recibirá el archivo de la imágen basada en distancia.

In [ ]:
# Importar librerias
import utils
carpeta_estilo = "/content/drive/MyDrive/S6_220521/Estilo" #@param {type:"string"}

# Nombre de las imagenes
imagen_BN = "Muehlenbeckia_mask.jpg" #@param {type:"string"}
imagen_DISTANCIA = "Muehlenbeckia_concat.jpg" #@param {type:"string"}

entrada = carpeta_estilo + "/" + imagen_BN
salida = carpeta_estilo + "/" + imagen_DISTANCIA

# Crear la imagen de distancia
utils.text_image_preprocessing(entrada, salida)

2. Se le dice a Colab en donde guardar la imagen final.

In [ ]:
# Importar la libreria os e ir a la carpeta de Estilo
import os
%cd '/content/drive/MyDrive/S6_220521/Estilo'

3. Se crea una imagen compuesta por el estilo original y la imagen de distancia

  - `montage` llama a la librería ImageMagick.
  - `-tile x1` establece que la imagen tenga solo una fila.
  - `-geometry 500x500` establece el tamaño de cada imagen.
  - `background none` establece que la imagen no tenga fondo o marco.

In [ ]:
!montage "/content/drive/MyDrive/S6_220521/Estilo/Muehlenbeckia_distance.png" "/content/drive/MyDrive/S6_220521/Estilo/Muehlenbeckia_original.png" -tile x1 -geometry 500x500 -background none Muehlenbeckia_concat.jpg

Luego de ejecutar los comandos quedará una imagen como la siguiente. **Esta será la imagen final de estilo:**

![concat](https://i.imgur.com/pbkmUVk.png)

---
# **ShapeMatchingGAN**

## Paso 1: Simplificación de estructura

Lo primero que hace SMGAN es simplificar la forma del estilo teniendo en cuenta la forma del texto.

**El resultado de este proceso es un modelo llamado *GB*.**

![paso_1](https://i.imgur.com/6lw1SwW.png)

---

Para ejecutar la siguiente celda tener encuenta los ajustes:

- `archivo_GB` define la ruta a la carpeta y el nombre de archivo del modelo GB. El nombre del archivo debe finalizar con **-GB.cpkt**.
- `carpeta_train` define la ruta a la carpeta donde están las imágenes de texto (de distancia) creadas anteriormente.
- `num_train_images` define cuántas de estas imágenes hay.
- `carpeta_augment` define la ruta a la carpeta **augment** que se encuentra en /ShapeMatchingGAN/data/rawtext.
- `num_augment_images` define cuántas de estas imagenes usar.
- `num_epochs` define el numero de iteraciones de entrenamiento.
- `batch_size` establece cuantas imágenes agrupa en cada paso del proceso.
- `num_BTraining` establece el numero de pasos en cada iteración. Durante el proceso este número se dividirá entre `batch_size`.


In [ ]:
archivo_GB = "/content/drive/MyDrive/S6_220521/Modelos/Muehlenbeckia-GB.cpkt" #@param {type:"string"}
carpeta_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train" #@param {type:"string"}
num_train_images =  50#@param {type : "integer"}
carpeta_augment = "/content/drive/MyDrive/S6_220521/ShapeMatchingGAN/data/rawtext/augment" #@param {type:"string"}
num_augment_images = 4 #@param {type:"slider", min:1, max:5, step:1}

num_epochs = 3 #@param {type:"slider", min:1, max:5, step:1}
batch_size = 8 #@param {type:"slider", min:4, max:16, step:4}
num_BTraining = 6400 #@param {type:"slider", min:400, max:12800, step:400}

from __future__ import print_function
import torch
from models import SketchModule
from utils import load_image, to_data, to_var, visualize, save_image, gaussian, weights_init
from utils import load_train_batchfnames, prepare_text_batch
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

opts = argparse.ArgumentParser()
opts.GB_nlayers = 6
opts.DB_nlayers = 5
opts.GB_nf = 32
opts.DB_nf = 32
opts.gpu = True

opts.epochs = num_epochs
opts.save_GB_name = archivo_GB
opts.batchsize = batch_size
opts.text_path = carpeta_train
opts.augment_text_path = carpeta_augment
opts.text_datasize = num_train_images
opts.augment_text_datasize = num_augment_images
opts.Btraining_num = num_BTraining

# create model
print('--- create model ---')
netSketch = SketchModule(opts.GB_nlayers, opts.DB_nlayers, opts.GB_nf, opts.DB_nf, opts.gpu)
if opts.gpu:
    netSketch.cuda()
netSketch.init_networks(weights_init)
netSketch.train()

print('--- training ---')
for epoch in range(opts.epochs):
    itr = 0
    fnames = load_train_batchfnames(opts.text_path, opts.batchsize, 
                                    opts.text_datasize, trainnum=opts.Btraining_num)
    fnames2 = load_train_batchfnames(opts.augment_text_path, opts.batchsize, 
                                    opts.augment_text_datasize, trainnum=opts.Btraining_num)
    for ii in range(len(fnames)):
        fnames[ii][0:opts.batchsize//2-1] = fnames2[ii][0:opts.batchsize//2-1]
    for fname in fnames:
        itr += 1
        t = prepare_text_batch(fname, anglejitter=True)
        t = to_var(t) if opts.gpu else t
        losses = netSketch.one_pass(t, [l//4.-1. for l in range(0,9)])      
        print('Epoch [%d/%d][%03d/%03d]' %(epoch+1, opts.epochs,itr,len(fnames)), end=': ')
        print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f'%(losses[0], losses[1], losses[2]))

print('--- save ---')
# directory
torch.save(netSketch.state_dict(), opts.save_GB_name)

### Pre-visualización del resultado de la simplificación:

**Importante:** Esta celda sólo funciona si se ejecuta inmediatamente después del entrenamiento. Si se cierra el cuaderno de Colab y se vuelve a abrir, no será posible pre-visualizar el resultado de la simplificación. Sin embargo, si el entrenamiento finalizó y se guardó el archivo _GB_ se puede continuar con el siguiente paso.

En `imagen_estilo` se debe colocar la ruta a la imágen final de estilo que contiene la versión de distancia y la versión original.

In [ ]:
imagen_estilo = "/content/drive/MyDrive/S6_220521/Estilo/Muehlenbeckia_concat.jpg" #@param {type:"string"}

netSketch.eval()
I = load_image(imagen_estilo)
I = to_var(I[:,:,:,0:I.size(3)//2])
result = netSketch(I, -1.)
visualize(to_data(result[0]))

## Paso 2: Transferencia de estructura

Lo siguiente que hace SMGAN es aprender a transferir la forma del estilo a las imágenes de texto.

**El resultado de este proceso es un modelo llamado *GS*.**

![paso_2](https://i.imgur.com/WUmXkVs.png)

---

Para ejecutar la siguiente celda tener encuenta los ajustes:

- `carpeta_modelos` define la ruta a la carpeta donde se guardó el archivo GB del paso anterior y donde se guardarán los demás modelos. **Se debe finalizar esta ruta con un /**
- `prefijo_modelos` define como se nombrarán los modelos. Se recomienda usar el mismo que se usó para el archivo GB.
- `archivo_GB` define la ruta al archivo GB creado en el paso anterior.
- `carpeta_train` define la ruta a la carpeta donde están las imágenes de texto (de distancia).
- `num_train_images` define cuántas de estas imágenes hay.
- `imagen_estilo` establece la ruta a la imagen final de estilo que contiene la versión de distancia y la versión original.
- `step_1`, `step_2`, `step_3` y `step_4` definen el numero de iteraciones en cada momento del entrenamiento.
- `num_STraining` establece el numero de pasos en cada iteración. Durante el proceso se dividirá entre 8.
- `preserve_glyph` define si preservar la forma del glifo o no.

In [ ]:
carpeta_modelos = "/content/drive/MyDrive/S6_220521/Modelos/" #@param {type:"string"}
prefijo_modelos = "Muehlenbeckia" #@param {type:"string"}
archivo_GB = "/content/drive/MyDrive/S6_220521/Modelos/Muehlenbeckia-GB.ckpt" #@param {type:"string"}
carpeta_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train" #@param {type:"string"}
num_train_images = 500 #@param {type : "integer"}
imagen_estilo = "/content/drive/MyDrive/S6_220521/Estilo/Muehlenbeckia_concat.jpg" #@param {type:"string"}

step_1 = 15 #@param {type:"slider", min:1, max:30, step:1}
step_2 = 30 #@param {type:"slider", min:1, max:60, step:1}
step_3 = 45 #@param {type:"slider", min:1, max:90, step:1}
step_4 = 5 #@param {type:"slider", min:1, max:10, step:1}

num_STraining = 1280 #@param {type:"slider", min:40, max:2560, step:40}

preserve_glyph = True #@param {type:"boolean"}

from __future__ import print_function
import torch
from models import SketchModule, ShapeMatchingGAN
from utils import load_image, to_data, to_var, visualize, save_image, gaussian, weights_init
from utils import load_train_batchfnames, prepare_text_batch, load_style_image_pair, cropping_training_batches
import random
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

opts = argparse.ArgumentParser()
# SMGAN
opts.GS_nlayers = 6
opts.DS_nlayers = 4
opts.GS_nf = 32
opts.DS_nf = 32
opts.GT_nlayers = 6
opts.DT_nlayers = 4
opts.GT_nf = 32
opts.DT_nf = 32

# SketchModule
opts.GB_nlayers = 6
opts.DB_nlayers = 5
opts.GB_nf = 32
opts.DB_nf = 32

opts.load_GB_name = archivo_GB

# train 
opts.gpu = True

opts.step1_epochs = step_1
opts.step2_epochs = step_2
opts.step3_epochs = step_3
opts.step4_epochs = step_4

opts.batchsize = 8

opts.Straining_num = num_STraining
opts.scale_num = 4
opts.Sanglejitter = True
opts.subimg_size = 256
opts.glyph_preserve = preserve_glyph
opts.text_datasize = num_train_images
opts.text_path = carpeta_train

# data and path
opts.save_path = carpeta_modelos
opts.save_name = prefijo_modelos
opts.style_name = imagen_estilo


# create model
print('--- create model ---')
netShapeM = ShapeMatchingGAN(opts.GS_nlayers, opts.DS_nlayers, opts.GS_nf, opts.DS_nf,
                 opts.GT_nlayers, opts.DT_nlayers, opts.GT_nf, opts.DT_nf, opts.gpu)
netSketch = SketchModule(opts.GB_nlayers, opts.DB_nlayers, opts.GB_nf, opts.DB_nf, opts.gpu)

if opts.gpu:
    netShapeM.cuda()
    netSketch.cuda()
netShapeM.init_networks(weights_init)
netShapeM.train()

netSketch.load_state_dict(torch.load(opts.load_GB_name))
netSketch.eval()

print('--- training ---')
# load image pair
scales = [l*2.0/(opts.scale_num-1)-1 for l in range(opts.scale_num)]
Xl, X, _, Noise = load_style_image_pair(opts.style_name, scales, netSketch, opts.gpu)
Xl = [to_var(a) for a in Xl] if opts.gpu else Xl
X = to_var(X) if opts.gpu else X
Noise = to_var(Noise) if opts.gpu else Noise
for epoch in range(opts.step1_epochs):
    for i in range(opts.Straining_num//opts.batchsize):
        idx = opts.scale_num-1
        xl, x = cropping_training_batches(Xl[idx], X, Noise, opts.batchsize, 
                                  opts.Sanglejitter, opts.subimg_size, opts.subimg_size)
        losses = netShapeM.structure_one_pass(x, xl, scales[idx])
        print('Step1, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.step1_epochs, i+1, 
                                                      opts.Straining_num//opts.batchsize), end=': ')
        print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lgly: %+.3f'%(losses[0], losses[1], losses[2], losses[3]))
netShapeM.G_S.myCopy()
for epoch in range(opts.step2_epochs):
    for i in range(opts.Straining_num//opts.batchsize):
        idx = random.choice([0, opts.scale_num-1])
        xl, x = cropping_training_batches(Xl[idx], X, Noise, opts.batchsize, 
                                  opts.Sanglejitter, opts.subimg_size, opts.subimg_size)
        losses = netShapeM.structure_one_pass(x, xl, scales[idx])
        print('Step2, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.step2_epochs, i+1, 
                                                      opts.Straining_num//opts.batchsize), end=': ')
        print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lgly: %+.3f'%(losses[0], losses[1], losses[2], losses[3]))
for epoch in range(opts.step3_epochs):
    for i in range(opts.Straining_num//opts.batchsize):
        idx = random.choice(range(opts.scale_num))
        xl, x = cropping_training_batches(Xl[idx], X, Noise, opts.batchsize, 
                                  opts.Sanglejitter, opts.subimg_size, opts.subimg_size)
        losses = netShapeM.structure_one_pass(x, xl, scales[idx])  
        print('Step3, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.step3_epochs, i+1, 
                                                      opts.Straining_num//opts.batchsize), end=': ')
        print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lgly: %+.3f'%(losses[0], losses[1], losses[2], losses[3]))
if opts.glyph_preserve:
    fnames = load_train_batchfnames(opts.text_path, opts.batchsize, 
                                    opts.text_datasize, opts.Straining_num)
    for epoch in range(opts.step4_epochs):
        itr = 0
        for fname in fnames:
            itr += 1
            t = prepare_text_batch(fname, anglejitter=False)
            idx = random.choice(range(opts.scale_num))
            xl, x = cropping_training_batches(Xl[idx], X, Noise, opts.batchsize, 
                                      opts.Sanglejitter, opts.subimg_size, opts.subimg_size)
            t = to_var(x) if opts.gpu else t
            losses = netShapeM.structure_one_pass(x, xl, scales[idx], t)  
            print('Step4, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.step4_epochs, itr+1, 
                                                      len(fnames)), end=': ')
            print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lgly: %+.3f'%(losses[0], losses[1], losses[2], losses[3])) 
        
print('--- save ---')
# directory
netShapeM.save_structure_model(opts.save_path, opts.save_name)

### Pre-visualización del resultado de la transferencia de estructura:

**Importante:** Esta celda sólo funciona si se ejecuta inmediatamente después del entrenamiento. Si se cierra el cuaderno de Colab y se vuelve a abrir, no será posible pre-visualizar el resultado de la transferencia de estructura. Sin embargo, si el entrenamiento finalizó y se guardó el archivo _GS_ se puede continuar con el siguiente paso.

En `imagen_train` se debe colocar la ruta a una de las imágenes de texto (de distancia).

In [ ]:
imagen_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train/0010.png" #@param {type:"string"}

netShapeM.eval()
I = load_image(imagen_train)
I = to_var(I[:,:,32:1000,32:1000])
I[:,0:1] = gaussian(I[:,0:1], stddev=0.2)
result = netShapeM.G_S(I, 1.0)
visualize(to_data(result[0]))

## Paso 3: Transferencia de textura

Por último, SMGAN aprende a transferir la textura de la imágen de estilo a la imágen de texto.

**El resultado de este proceso es un modelo llamado *GT*.**

![paso_3](https://i.imgur.com/QG41tLG.png)

---

Para ejecutar la siguiente celda tener encuenta los ajustes:

- `carpeta_modelos` define la ruta a la carpeta donde se guardaron los modelos en el paso anterior. **Finalizar esta ruta con un /**
- `prefijo_modelos` define como se nombrarán los modelos. Se recomienda usar el mismo que los archivos del paso anterior.
- `archivo_GS` define la ruta al archivo GS creado en el paso anterior.
- `carpeta_train` define la ruta a carpeta donde están las imágenes de texto (de distancia).
- `num_train_images` define cuántas de estas imágenes hay.
- `carpeta_augment` define la ruta a la carpeta **augment** que se encuentra en /ShapeMatchingGAN/data/rawtext.
- `num_augment_images` define cuántas de estas imagenes usar.
- `imagen_estilo` establece la ruta a la imagen final de estilo que contiene la versión de distancia y la versión original.
- `step_1` define el numero de iteraciones de entrenamiento.
- `num_TTraining` establece el numero de pasos en cada iteración. Durante el proceso se dividirá entre 4.

In [ ]:
carpeta_modelos = "/content/drive/MyDrive/S6_220521/Modelos/" #@param {type:"string"}
prefijo_modelos = "Muehlenbeckia" #@param {type:"string"}
archivo_GS = "/content/drive/MyDrive/S6_220521/Modelos/Muehlenbeckia-GS.ckpt" #@param {type:"string"}

carpeta_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train" #@param {type:"string"}
num_train_images =  1024#@param {type : "integer"}
carpeta_augment = "/content/drive/MyDrive/S6_220521/ShapeMatchingGAN/data/rawtext/augment" #@param {type:"string"}
num_augment_images = 1 #@param {type:"slider", min:1, max:5, step:1}

imagen_estilo = "/content/drive/MyDrive/S6_220521/Estilo/Muehlenbeckia_concat.jpg" #@param {type:"string"}

step_1 = 40 #@param {type:"slider", min:1, max:80, step:1}

num_TTraining = 600 #@param {type:"slider", min:4, max:1200, step:4}

from __future__ import print_function
import torch
from models import SketchModule, ShapeMatchingGAN
from utils import load_image, to_data, to_var, visualize, save_image, gaussian, weights_init
from utils import load_train_batchfnames, prepare_text_batch, load_style_image_pair, cropping_training_batches
import random
from vgg import get_GRAM, VGGFeature
import torchvision.models as models
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

opts = argparse.ArgumentParser()
# SMGAN
opts.GS_nlayers = 6
opts.DS_nlayers = 4
opts.GS_nf = 32
opts.DS_nf = 32
opts.GT_nlayers = 6
opts.DT_nlayers = 4
opts.GT_nf = 32
opts.DT_nf = 32

# train 
opts.gpu = True

opts.texture_step1_epochs = step_1
opts.texture_step2_epochs = 10

opts.batchsize = 4

opts.Ttraining_num = num_TTraining 

opts.Tanglejitter = True
opts.subimg_size = 256
opts.style_loss = False

opts.text_path = carpeta_train
opts.text_datasize = num_train_images
opts.augment_text_path_path = carpeta_augment
opts.augment_text_datasize = num_augment_images


# data and path
opts.save_path = carpeta_modelos
opts.save_name = prefijo_modelos
opts.style_name = imagen_estilo
opts.load_GS_name = archivo_GS

# create model
print('--- create model ---')
netShapeM = ShapeMatchingGAN(opts.GS_nlayers, opts.DS_nlayers, opts.GS_nf, opts.DS_nf,
                 opts.GT_nlayers, opts.DT_nlayers, opts.GT_nf, opts.DT_nf, opts.gpu)

if opts.gpu:
    netShapeM.cuda()
netShapeM.init_networks(weights_init)
netShapeM.train()

if opts.style_loss:
    netShapeM.G_S.load_state_dict(torch.load(opts.load_GS_name))  
    netShapeM.G_S.eval()
    VGGNet = models.vgg19(pretrained=True).features
    VGGfeatures = VGGFeature(VGGNet, opts.gpu)
    for param in VGGfeatures.parameters():
        param.requires_grad = False
    if opts.gpu:
        VGGfeatures.cuda()
    style_targets = get_GRAM(opts.style_name, VGGfeatures, opts.batchsize, opts.gpu)
    
print('--- training ---')
# load image pair
_, X, Y, Noise = load_style_image_pair(opts.style_name, gpu=opts.gpu)
Y = to_var(Y) if opts.gpu else Y
X = to_var(X) if opts.gpu else X
Noise = to_var(Noise) if opts.gpu else Noise
for epoch in range(opts.texture_step1_epochs):
    for i in range(opts.Ttraining_num//opts.batchsize):
        x, y = cropping_training_batches(X, Y, Noise, opts.batchsize, 
                                  opts.Tanglejitter, opts.subimg_size, opts.subimg_size)
        losses = netShapeM.texture_one_pass(x, y)
        print('Step1, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.texture_step1_epochs, i+1,
                                                     opts.Ttraining_num/opts.batchsize), end=': ')
        print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lsty: %+.3f'%(losses[0], losses[1], losses[2], losses[3])) 
if opts.style_loss:
    fnames = load_train_batchfnames(opts.text_path, opts.batchsize, 
                                    opts.text_datasize, trainnum=opts.Ttraining_num)
    for epoch in range(opts.texture_step2_epochs):
        itr = 0
        for fname in fnames:
            itr += 1
            t = prepare_text_batch(fname, anglejitter=False)
            x, y = cropping_training_batches(X, Y, Noise, opts.batchsize, 
                                  opts.Tanglejitter, opts.subimg_size, opts.subimg_size)
            t = to_var(t) if opts.gpu else t
            losses = netShapeM.texture_one_pass(x, y, t, 0, VGGfeatures, style_targets)  
            print('Step2, Epoch [%02d/%02d][%03d/%03d]' %(epoch+1, opts.texture_step2_epochs, 
                                                         itr, len(fnames)), end=': ')
            print('LDadv: %+.3f, LGadv: %+.3f, Lrec: %+.3f, Lsty: %+.3f'%(losses[0], losses[1], losses[2], losses[3])) 
        
print('--- save ---')
# directory
netShapeM.save_texture_model(opts.save_path, opts.save_name)

### Pre-visualización del resultado de la transferencia de textura:

**Importante:** Esta celda sólo funciona si se ejecuta inmediatamente después del entrenamiento. Si se cierra el cuaderno de Colab y se vuelve a abrir, no será posible pre-visualizar el resultado de la transferencia de textura. Sin embargo, si el entrenamiento finalizó y se guardó el archivo _GT_ se puede continuar con el siguiente paso.

- En `archivo_GS` se define la ruta al archivo *GS* creado en el paso 2.
- En `imagen_train` se establece la ruta a una de las imágenes de texto (de distancia).
- `nombre_archivo_salida` define la ruta de la carpeta a donde se guardará la imágen de pre-visualización. **Se debe finalizar con /ElNombreDelArchivo.png**

In [ ]:
archivo_GS = "/content/drive/MyDrive/S6_220521/Modelos/Muehlenbeckia-GS.ckpt" #@param {type:"string"}
imagen_train = "/content/drive/MyDrive/S6_220521/Glifos/base/train/0050.png" #@param {type:"string"}
nombre_archivo_salida = "/content/drive/MyDrive/S6_220521/Results/0.png" #@param {type:"string"}

netShapeM.G_S.load_state_dict(torch.load(archivo_GS))  
netShapeM.eval()
I = load_image(imagen_train)
I = to_var(I[:,:,32:1000,32:1000])
result = netShapeM(I, 1)
visualize(to_data(result[0]))
save_image(to_data(result[0]), nombre_archivo_salida)

## Paso 4: Usar los modelos resultantes para generar los injertos:

En la siguiente celda se usan los modelos resultantes para crear imágenes con diferentes niveles de estilización y deformación.

- `!python3 ...` define la ruta al archivo `test_InjertosTipograficos.py` que se encuentra en la carpeta /ShapeMatchingGAN/src.
- `--text_name ...` define la imagen de texto que se quiere estilizar. Debe ser una imágen de distancia preferiblemente.
- `--scale` define el grado de estilización. Se puede usar `-3, -2 y -1`, siendo -3 el mayor grado de estilización.
- `--scale_step` define cuantos pasos hacer para llegar de 0 al valor que se use en `--scale`. Entre más pequeño el valor, más pasos se harán y más suave será la transición. Se pueden usar valores entre 0.01 y 0.5.

**Importante: entre mayor sea el número en `--scale` y menor en `--scale_steps`, por ejemplo `-3 y 0.01`, el número de imágenes que se crean aumentará.**

- `--structure_model ...` define la ruta al archivo **GS** creado en la transferencia de estructura.
- `--texture_model ...` define la ruta al archivo **GT** creado en la transferencia de textura.
- `--result_dir ...` define la ruta a la carpeta donde se guardarán las imágenes.
- `--name ...` establece que nombre usar para cada imagen.
- `--gpu` le dice a Colab que utilice la Tarjeta de Video.

In [ ]:
!python3 "/content/drive/MyDrive/S6_220521/ShapeMatchingGAN/src/test_InjertosTipograficos.py" \
--text_name "/content/drive/MyDrive/S6_220521/Glifos/base/train/0000.png" \
--scale -3 --scale_step 0.025 \
--structure_model "/content/drive/MyDrive/S6_220521/Modelos//Muehlenbeckia-GS.ckpt" \
--texture_model "/content/drive/MyDrive/S6_220521/Modelos//Muehlenbeckia-GT.ckpt" \
--result_dir "/content/drive/MyDrive/S6_220521/Modelos/Results/A" --name "Muehlenbeckia_A" \
--gpu


### Visualización de una de las imágenes:

Para ver alguna de las imágenes creadas se puede utilizar la siguiente celda. Se debe cambiar la ruta a la imagen escogida.

In [ ]:
from IPython.display import Image
Image("/content/drive/MyDrive/S6_220521/Results/A/Muehlenbeckia_A_0.png")

### Generación de video

Si se quiere realizar una versión animada del injerto usando las imágenes creadas en el paso 4, es posible realizarlo con la herramienta [ffmpeg](https://ffmpeg.org/). Es un software gratuiro que sirve para grabar, convertir, codificar y transmitir video. Como ImageMagick, se utiliza con comandos.

Lo primero que se debe hacer es instalar la herramienta:

In [ ]:
!sudo apt-get update && sudo apt-get upgrade && apt install ffmpeg

Luego se debe ir a la carpeta que contiene las imágenes y renombrarlas para que sigan un patrón `0000.png, 0001.png, 0002.png ...` y no se generen saltos. **Se debe cambiar la ruta en la linea que comienza con `os.chdir()`.**

In [ ]:
# Importa las librerías necesarias
import os
import glob

# Ruta a la carpeta con las imágenes creadas anteriormente. No olvidar cambiar.
os.chdir(r"/content/drive/MyDrive/S6_220521/Results/A")
for index, oldfile in enumerate(glob.glob("*.png"), start=0):
    newfile = '{:04}.png'.format(index)
    os.rename (oldfile,newfile)

Finalmente con el siguiente comando se crea un video usando los frames de la carpeta.

- `!ffmpeg` llama la herramienta.
- `loop 1` define que el video sea un loop.
- `-framerate 30` define los fotogramas por segundo.
- `pattern_type glob -i '*.png'` define que ffmpeg use todos los archivos con extensión .png.
- `-t 5` define la duración del video en segundos.
- `-c:v libx264` define que se use el códec de video [h.264](https://trac.ffmpeg.org/wiki/Encode/H.264).
- `-pix_fmt yuv420p` define se use un [submuestreo](https://es.wikipedia.org/wiki/Submuestreo_de_crominancia) 4:2:0.

- `out.mp4` define el nombre del archivo de salida.

In [ ]:
!ffmpeg -loop 1 -framerate 30 -pattern_type glob -i '*.png' -t 5 -c:v libx264 -pix_fmt yuv420p out.mp4